In [1]:
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder  # On convertit les labels en nombres entiers
import time
import pandas as pd


In [3]:

anime_data=pd.read_csv("anime_data.csv")

anime_data=anime_data.drop(columns=["ranking", "rating", "title", "source", "media_type", "status", "nsfw"])

starting_time = time.time()

In [4]:
# Classement des différents types de popularités en 4 types distincts :

def categorize_popularity(popularity):
    if popularity < 50:
        return 3  # Très populaire
    elif 50 <= popularity < 200:
        return 2  # Populaire
    elif 200 <= popularity < 500:
        return 1  # Moyen
    elif 500 <= popularity < 1000:
        return 0  # Nul
    return 0  # Si aucune catégorie ne correspond (valeurs > 1000 ou autres cas)

In [5]:
# Création de la nouvelle colonne categorize_popularity 
anime_data['popularity_category'] = anime_data['popularity'].apply(categorize_popularity)


# Préparation des exemples et des étiquettes
X = anime_data.drop(columns=['popularity', 'popularity_category'])  # Autres caractéristiques, sans 'popularity'
y = anime_data['popularity_category']  # Variable dépendante (nouvelle catégorisation)

In [ ]:
# Séparation des données en test/train
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Assurer que toutes les classes sont représentées dans y_train et y_test
y_train_classes = y_train.unique()
y_test_classes = y_test.unique()

# Si des classes sont absentes, on peut en ajouter une ligne pour traiter cela
if len(set(y_train_classes) - set(y_test_classes)) > 0:
    y_test = pd.concat([y_test, pd.Series([y_train_classes[0]] * (len(y_train_classes) - len(y_test_classes)), index=y_test.index)])

y_test=y_test.reset_index(drop=True)

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_test)

(11552, 8)
(11552,)
(2888, 8)
(2888,)
       num_scoring_users   rank  num_episodes  average_episode_duration  \
4417               19203   4435            12                      1481   
907                14170    908             0                      1430   
2859              189058   2869            13                       210   
5836                2433   5868             2                      1730   
2715               22783   2725             1                      6266   
...                  ...    ...           ...                       ...   
8791                2351   8901            13                      1466   
9144                 361   9268             1                      5952   
11350                147  11677            13                       240   
1210              429290   1211            13                      1440   
8086                 625   8159            35                      1400   

          id  num_list_users  mean  start_year  
4417    3616

In [8]:
# Training time
training_time = time.time()-starting_time
print("Trainning time:", training_time)

Trainning time: 27.45297908782959


In [9]:
# Modèle XGBoost et entrainement du modèle
model = xgb.XGBClassifier(
    objective='multi:softmax',  #classification mutliclasses
    num_class=len(y.unique()),
    use_label_encoder=False,
    eval_metric='mlogloss'  #perte log
)

In [11]:
model.fit(X_train, y_train)

y_pred=model.predict(X_test.transpose())
print(classification_report(y_test, y_train))

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [08:22:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179675237/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


ValueError: feature_names mismatch: ['num_scoring_users', 'rank', 'num_episodes', 'average_episode_duration', 'id', 'num_list_users', 'mean', 'start_year'] ['4417', '907', '2859', '5836', '2715', '7261', '10586', '8471', '2405', '10723', '9590', '8973', '5957', '8653', '8628', '13318', '2039', '10290', '13469', '14188', '13673', '8145', '273', '10335', '13686', '6958', '3079', '4393', '1598', '6576', '5107', '4148', '2796', '11413', '8620', '6148', '4315', '3322', '1121', '8137', '169', '6195', '542', '8781', '3519', '2813', '4921', '6108', '11086', '13401', '8994', '13615', '7337', '13419', '3116', '3288', '2254', '1807', '11611', '4028', '12848', '13107', '9997', '357', '10746', '13944', '11007', '13651', '11857', '9431', '5418', '7995', '14360', '12163', '7553', '2019', '7247', '9825', '14312', '10072', '12084', '12014', '6415', '7306', '8913', '11875', '9967', '10388', '10514', '8664', '4838', '6951', '7953', '10299', '2034', '6270', '13359', '2021', '5723', '9568', '9927', '518', 

In [ ]:
# AUC Test
y_pred_proba = model.predict_proba(X_test)  # Prédictions sous forme de probabilités
auc_test = roc_auc_score(y_true=y_test, y_score=y_pred_proba, multi_class='ovr')
print("auc_test_xgb", auc_test)

# Modèle de classification binaire - transformers et NLP